#### Week 3: Building Advanced RAG Applications.  Authored by Chris Sanchez.

# Week 3 - Notebook 7 --> Context Enrichment

# Background  
---
The background material for this topic/notebook is covered in the [Week 3 Course Content](https://uplimit.com/course/rag-applications/session/session_clzlsa20a01di197e4tij7vgm/module/module_clzlsa9a702cy1dm671cg0xr0) section titled **Context Enrichment**.  

# Overview
---
The concept of Context Enrichment became popularized with the advent of LLMs that could reason over ever-increasing context window sizes. Prior to 2022 most open-source Reader models were limited to a 512 token context window. From a vector search perspective, adding the Context Enrichment technique to your toolbox allows the best of both worlds, because you can figure out which chunk size works best for your embedding model/reranker combo and then expand the retrieved text chunk with surrounding context so that the Reader LLM has the additional context it needs to answer the user query. Though not the only reason, this technique is partly why setting the chunk overlap parameter to zero (when chunking your text into sentences) is a good call.  

This notebook will walk you through the process of creating an `expanded_content` field that you can add to an existing dataset, which can then be indexed onto your Weaviate cluster. 
- No need to create a new dataset, simpy use a prexisting dataset (i.e. `huberman_minilm_256.parquet`)
- Group dataset episodes together by `video_id`.  Performing this step will ensure that all before and after text chunks are all from the same episode and there is no "bleed-over" into another episode.
- Loop over each set of episode chunks and join pre-, current, and post- chunks together as a single string.  The window size can be set as a parameter.
- Join each chunk to the original dataset as an additional `expanded_content` field.
- Either index the new dataset on a new collection, or update an existing collection.  The properties file used to create the index schema on Weaviate already includes an `expanded_content` property. 

In [1]:
import sys
sys.path.append('../')

In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True)
import os

In [3]:
from src.database.properties_template import properties
from src.database.weaviate_interface_v4 import WeaviateWCS
from src.preprocessor.preprocessing import FileIO
from sentence_transformers import SentenceTransformer
from rich import print
from tqdm import tqdm
import tiktoken

/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Load Data
No need to create a new dataset, simply use the data that you already have. 

In [4]:
data_path = "./huberman-minilmL6-256.parquet" #'../data/huberman_minilm-256.parquet'
data = FileIO.load_parquet(data_path)

Shape of data: (23123, 13)
Memory Usage: 2.29+ MB


### Create Expanded Content

In [5]:
from itertools import groupby

def groupby_episode(data: list[dict], key_field: str='video_id') -> list[list[dict]]:
    '''
    Separates entire podcast corpus into individual 
    lists of discrete episodes.
    '''
    episodes = []
    for _, group in groupby(data, lambda x: x[key_field]):
        episode = [chunk for chunk in group]
        episodes.append(episode)
    return episodes

In [6]:
def create_expanded_content(data: list[dict] | list[list[str]],
                            window_size: int=1,
                            num_episodes: int=193,
                            key_field: str='video_id'
                            ) -> list[list[str]]:
    '''
    Creates expanded content from original chunks of text, for use with 
    expanded content retrieval.  Takes in raw data as a list of dictionaries 
    or accepts a list of chunked episodes already grouped. 
    
    Window size sets the number of chunks before and after the original chunk.  
    For example a window_size of 2 will return five joined chunks.  2 chunks 
    before original chunk, the original, and 2 chunks after the original.  
    
    Expanded content is grouped by podcast episode, and chunks are assumed 
    to be kept in order by which they will be joined as metadata in follow-on 
    processing.
    '''
    # check if dictionaries or already grouped episodes is being passed in
    if isinstance(data[0], dict):
        # groupby data into episodes using video_id key
        episodes = groupby_episode(data, key_field)
        assert len(episodes) == num_episodes, f'Number of grouped episodes does not equal num_episodes ({len(episodes)} != {num_episodes})'

        # extract content field and ensure episodes maintain their grouping
        chunk_list = [[d['content'] for d in alist] for alist in episodes]
        
    expanded_contents = []
    for episode in tqdm(chunk_list):
        episode_container = []
        for i, chunk in enumerate(episode):
            start = max(0, i-window_size)
            end = i+window_size+1
            expanded_content = ' '.join(episode[start:end])
            episode_container.append(expanded_content)
        expanded_contents.append(episode_container)
    return expanded_contents

# Assignment 3.1 - 
***
#### *Create Expanded Content chunks and join them to existing data*

#### INSTRUCTIONS
1. Execute the `create_expanded_content` function.  Depending on your chunk size, it is likely best to use the default window size of 1.  Meaning, 1 chunk of text will be added before and after the original text chunk, for a total of three chunks for each `expanded_content` field.
2. Assuming you are going to join the data back to the original dataset from which it came, you'll need to flatten out the list of episode into a single list of text chunks.
3. Write a function that updates your original dataset with the new expanded content by updating the dataset with an `expanded_content` key. 

In [ ]:
########################
# START YOUR CODE HERE #
########################

expanded_content = create_expanded_content(data, 1)
flattened_content = [item for items in expanded_content for item in items]

# azimuth check to ensure you're heading in the right direction
flat_length = len(flattened_content)
data_length = len(data)
assert flat_length == data_length, 'Mismatch in lengths. Double check how you flattened your expanded_content'

def join_expanded_content(data: list[dict],
                          flattened_content: list[list[str]],
                          new_key: str = 'expanded_content'
                          ) -> list[dict]:
    '''
    Updates data with an expanded_content key.
    '''

    for i, chunk in enumerate(data):
        chunk[new_key] = flattened_content[i]

########################
# END YOUR CODE HERE #
########################
    
    return data

data = join_expanded_content(data, flattened_content)

100%|██████████| 193/193 [00:00<00:00, 2579.70it/s]


#### After executing the above function, run the following cells as a post-check

In [8]:
#ensure all expanded content fields are present
for d in data:
    assert d.get('expanded_content', -1) != -1
    
#compare your initial result with the following cell
print(data[0])

{
    'video_id': '-OBCwiPPfEU',
    'title': 'Dr. Matthew Walker: The Biology of Sleep & Your Unique Sleep Needs | Huberman Lab Guest Series',
    'length_seconds': 10773,
    'keywords': [
        'andrew huberman',
        'huberman lab podcast',
        'huberman podcast',
        'dr. andrew huberman',
        'neuroscience',
        'huberman lab',
        'andrew huberman podcast',
        'the huberman lab podcast',
        'science podcast'
    ],
    'view_count': 162689,
    'episode_num': 193,
    'episode_url': 'https://www.youtube.com/watch?v=-OBCwiPPfEU',
    'thumbnail_url': 'https://i.ytimg.com/vi/-OBCwiPPfEU/mqdefault.jpg',
    'guest': 'Dr. Matthew Walker',
    'summary': "In the Huberman Lab podcast episode featuring Dr. Matthew Walker, the discussion centers on the 
biology of sleep and how it impacts mental and physical health. They delve into the different stages of sleep, 
including non-REM and REM sleep, and their unique roles in brain function, such as memory consolidation and 
emotional regulation. They also explore practical strategies for improving sleep quality, such as managing light 
exposure, temperature, and timing of activities. Dr. Walker introduces the QQRT (Quality, Quantity, Regularity, and
Timing) formula for identifying individual sleep needs. The conversation highlights the detrimental effects of 
sleep deprivation on hormones, metabolism, and the risk of diseases like diabetes and Alzheimer's. They touch on 
the evolutionary importance of sleep, suggesting that its universal presence among species indicates a vital 
function for survival.",
    'doc_id': '-OBCwiPPfEU_0',
    'content': "Welcome to the Huberman Lab guest series, where I and an expert guest discuss science and 
science-based tools for everyday life. I'm Andrew Huberman, and I'm a professor of neurobiology and ophthalmology 
at Stanford School of Medicine. Today's episode marks the first in our six-episode series all about sleep. Our 
expert guest for this series is Dr. Matthew Walker, professor of neuroscience and psychology and the director of 
the Center for Sleep Science at the University of California, Berkeley. He is also the author of the bestselling 
book, Why We Sleep. During the course of the six-episode series, for which we release one episode per week, 
starting with this episode one, we cover essentially all aspects of sleep and provide numerous practical tools to 
improve your sleep. For instance, we discuss the biology of sleep, including the different sleep stages, as well as
why sleep is so important for our mental and physical health. We also talk about how sleep regulates things like 
emotionality and learning and neuroplasticity, that is your brain's ability to change in response to experience. 
And we discuss the various things that you can do to improve your sleep.",
    'content_embedding': [
        -0.022558223456144333,
        -0.02126704715192318,
        0.05489971116185188,
        0.07128091156482697,
        0.06499785929918289,
        0.05991667881608009,
        -0.01999969780445099,
        0.0005336769390851259,
        0.029837189242243767,
        -0.011097035370767117,
        -0.12040792405605316,
        0.0071654245257377625,
        -0.03563549742102623,
        0.023756450042128563,
        -0.048562925308942795,
        -0.01306042168289423,
        -0.015637293457984924,
        -0.02250908873975277,
        0.018875697627663612,
        0.03482157737016678,
        0.029338020831346512,
        0.0023340745829045773,
        0.1370449662208557,
        -0.007225439418107271,
        -0.0004594569618348032,
        0.041375886648893356,
        0.033691249787807465,
        -0.062393106520175934,
        -0.007559778168797493,
        -0.06433174759149551,
        -0.016501760110259056,
        0.05270547419786453,
        0.012666643597185612,
        -0.049873560667037964,
        0.009051398374140263,
        0.010008126497268677,
        0.007533654570579529,
        0.0

<details> 
    <summary>
        Click to compare your results from the cell above with the following:
</summary>  
    
```
"Welcome to the Huberman Lab guest series, where I and an expert guest discuss science and science-based tools for everyday life. I'm Andrew Huberman, and I'm a professor of neurobiology and ophthalmology at Stanford School of Medicine. Today's episode marks the first in our six-episode series all about sleep. Our expert guest for this series is Dr. Matthew Walker, professor of neuroscience and psychology and the director of the Center for Sleep Science at the University of California, Berkeley. He is also the author of the bestselling book, Why We Sleep. During the course of the six-episode series, for which we release one episode per week, starting with this episode one, we cover essentially all aspects of sleep and provide numerous practical tools to improve your sleep. For instance, we discuss the biology of sleep, including the different sleep stages, as well as why sleep is so important for our mental and physical health. We also talk about how sleep regulates things like emotionality and learning and neuroplasticity, that is your brain's ability to change in response to experience. And we discuss the various things that you can do to improve your sleep. Everything from how to time lighting, temperature, exercise, eating, and the various things that can impact sleep both positively and negatively, such as alcohol, cannabis, and various supplements and drugs that have been shown to improve sleep. We also talk about naps, dreaming and the role of dreams, and lucid dreaming, which is when you dream and you are aware that you are dreaming. In today's episode one, we specifically focus on why sleep is so important and what happens when we do not get enough sleep or enough quality sleep. We also talk about the various sleep stages, and we also talk about a very specific formula that everyone should know for themselves called QQRT, which is an acronym that stands for quality, quantity, regularity, and timing of sleep. Four factors which today you'll learn how to identify specifically for you what your optimal QQRT is, and then to apply that in order to get the best possible night's sleep, which of course equates to the best possible level of focus and alertness throughout your days. Both Dr."
```
</details>

### Index the Data
---
You have two options here:
1. Easy way: Simply index the data on a new Collection.
2. Hard way: Read all existing uuids on current Collection and update each object by linking the doc_ids.

As mentioned earlier, the `expanded_content` property is already part of the index configuration of properties.  See the last property entry after printing the `properties` variable: 

In [9]:
print(properties)

[
    Property(
        name='video_id',
        dataType=<DataType.TEXT: 'text'>,
        description='Unique identifier of the video episode',
        indexFilterable=True,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='title',
        dataType=<DataType.TEXT: 'text'>,
        description='Title of the video episode',
        indexFilterable=True,
        indexSearchable=True,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='length_seconds',
        dataType=<DataType.INT: 'int'>,
        description='Length of the video episode in seconds',
        indexFilterable=True,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='thumbnail_url',
        dataType=<DataType.TEXT: 'text'>,
        description='URL of the video episode thumbnail picture',
        indexFilterable=False,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='keywords',
        dataType=<DataType.TEXT_ARRAY: 'text[]'>,
        description='Keywords associated with the video episode',
        indexFilterable=True,
        indexSearchable=True,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='summary',
        dataType=<DataType.TEXT: 'text'>,
        description='Short summary of the episode content',
        indexFilterable=False,
        indexSearchable=True,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='view_count',
        dataType=<DataType.INT: 'int'>,
        description='Number of views the video episode has received',
        indexFilterable=True,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='episode_num',
        dataType=<DataType.INT: 'int'>,
        description='User-defined episode number in the series',
        indexFilterable=True,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='doc_id',
        dataType=<DataType.TEXT: 'text'>,
        description='Unique document identifier created during preprocessing of documents',
        indexFilterable=True,
        indexSearchable=False,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='content',
        dataType=<DataType.TEXT: 'text'>,
        description='Chunk of text from the video episode',
        indexFilterable=False,
        indexSearchable=True,
        indexRangeFilters=None,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='guest',
        dataType=<DataType.TEXT: 'text'>,
        description='Guests featured in the video episode',
        indexFilterable=True,
        indexSearchable=True,
        indexRangeFilters=None,
     

In [10]:
from src.database.weaviate_interface_v4 import WeaviateIndexer

#read env vars from local .env file
api_key = os.environ['WEAVIATE_API_KEY']
url = os.environ['WEAVIATE_ENDPOINT']
model_path = 'sentence-transformers/all-MiniLM-L6-v2'

#instantiate client
client = WeaviateWCS(endpoint=url, api_key=api_key, model_name_or_path=model_path, skip_init_checks=True)

#example of using the private _client attribute
client._client.is_connected()

collection_name = 'Huberman_minilm_256_expandedContent'

client.create_collection(collection_name=collection_name, properties=properties, description='Huberman Labs: 193 full-length transcripts')

indexer = WeaviateIndexer(client)

batch_object = indexer.batch_index_data(data, collection_name)

total_docs = client.get_doc_count(collection_name)

Collection "Huberman_minilm_256_expandedContent" created


100%|██████████| 23123/23123 [03:55<00:00, 98.19it/s] 


Processing finished in 3.93 minutes.
Batch job completed with zero errors.
Found 23123 documents in collection "Huberman_minilm_256_expandedContent"


In [12]:
display_properties = [prop.name for prop in client.show_collection_properties(collection_name)]

In [14]:


query="What can I do to increase both my healthspan and lifespan"
response = client.keyword_search(request=query,
                                 collection_name=collection_name,
                                 limit=5,
                                 return_properties=display_properties,
                                 filter=None,       # filtering is discussed as an optional final part of this notebook
                                 return_raw=False)  # turn this flag on and off to see how the responses are being reformatted

print(response)

[
    {
        'guest': 'Dr. Paul Conti',
        'content': "And if you think about, we're talking about relationships. So I would like my relationships to 
last as long as possible. That means now we're talking about my lifespan and my healthspan. That then becomes part 
of that discussion. And of course, we're very interested in lifespan and healthspan. And we see people do much, 
much better when they're interconnected in the world around them, when they're still learning new things. We know 
that's true, that people often will sort of trail off of what they're learning, whether it's music, it's 
literature, it's the world around us, relatively early in our lifespan. But the person who's interconnected, 
learning new things, has a much greater probability of living longer and living healthier. So? So I think what's 
that about? It's about a generative drive, right? It's about, you know what? I've learned a lot of things over the 
course of my life and I'm 80 years old now and like, great that there's more to learn, right? And like, that's what
you see. It makes me think of a woman who's around 90 years old in my practice. She's always learning new things. 
Like's super interested in things.",
        'episode_url': 'https://www.youtube.com/watch?v=eMqWH3LYiII',
        'summary': "In the context of romantic relationships, Dr. Paul Conti explains that the generative drive is 
the key to healthy dynamics, not necessarily common interests or backgrounds. A strong generative drive in both 
individuals allows for open communication, mentalizing each other's emotional states, and approaching each other 
with agency and gratitude. This drive enables partners to navigate differences in desires or needs, such as varying
sex drives, by finding a middle ground that satisfies both parties without feeling compromised. Conti emphasizes 
that relationships thrive when individuals are proactive and grateful, leading to a more generative and fulfilling 
partnership. Moreover, he suggests that love can indeed overcome obstacles when both individuals have a strong 
generative drive, allowing the relationship itself to become a new entity with its own emergent properties.",
        'thumbnail_url': 'https://i.ytimg.com/vi/eMqWH3LYiII/mqdefault.jpg',
        'keywords': [
            'andrew huberman',
            'huberman lab podcast',
            'huberman podcast',
            'dr. andrew huberman',
            'neuroscience',
            'huberman lab',
            'andrew huberman podcast',
            'the huberman lab podcast',
            'science podcast',
            'dr. paul conti',
            'stanford school of medicine',
            'harvard medical school',
            'trauma: the invisible epidemic',
            'mental health',
            'healthy relationships',
            'romantic relationships',
            'work relationships',
            'colleague relationships',
            'friendships',
            'conscious mind',
            'unconscious mind',
            'conflict resolution',
            'healthy boundaries'
        ],
        'title': 'Dr. Paul Conti: How to Build and Maintain Healthy Relationships | Huberman Lab Guest Series',
        'view_count': 497614,
        'video_id': 'eMqWH3LYiII',
        'episode_num': 156,
        'expanded_content': "And I'm also sensing that the words like-minded, you know, like-minded people has so 
much more to do with their generative drives and how those match up as opposed to the activities that they prefer 
engaging in and the sorts of foods they like or the movies they like. And which makes sense at some level, but I 
think it is still counterintuitive for a lot of us who just kind of reflexively think, oh, like they like the same 
things, you know, or maybe even they met at work because they like the same type of work. They like to live in a 
certain area of the country and therefore by proximity they met. And this raises all sorts of interesting ideas, 
about th

## Conclusion
---
After you've indexed the data you will now have a way to retrieve content on a fine-grained level, and provide your LLM Reader with an expanded context.  You will be able to see this in action when you add `expanded_content` as a `return_property` in your Streamlit UI. 🎉  

However, the important idea here is how does adding expanded content affect your RAG system performance?  Assuming you completed Notebook 5 you now have the means at your disposal of running an evaluation with the Reader LLM ingesting the `expanded_content` field as context instead of the `content` field. Setting up your own evaluation will be a good exercise in understanding how all of these pieces are put together.  You'll note that the function used to build the user message at run time `generate_prompt_series` as well as the  `create_context_blocks` function have a `content_key` parameter.  You will want to pass in `expanded_content` as an argument here instead of `content`. 

## OPTIONAL: Update an existing Collection
---
For those interested in doing things the hard way here is some starter code.  No guarantee that this code will work as written, but it gives you the idea of how you would accomplish this task; or just create a new Collection... 😀:

In [19]:
# get collection object
api_key = os.environ['WEAVIATE_API_KEY']
endpoint = os.environ['WEAVIATE_ENDPOINT']

client = WeaviateWCS(endpoint, api_key)

collection_name = 'Huberman_minilm_256'
collection = client._client.collections.get(collection_name)

In [20]:
# This step will take a few minutes to read every object id on the Weaviate cluster
doc_id_cache = {item.properties['doc_id']:item.uuid for item in tqdm(collection.iterator())}

23123it [00:51, 452.85it/s]


`doc_id_cache` example:
```
{'-OBCwiPPfEU_8': _WeaviateUUIDInt('018455e9-47ab-41cc-b592-c431fd8df75f'),
 '-OBCwiPPfEU_4': _WeaviateUUIDInt('03a18709-0334-4f18-9375-4a8a3f162cfb'),
 '-OBCwiPPfEU_0': _WeaviateUUIDInt('0da24442-3263-46d7-91af-ef2a34d27a9c'),
 '-OBCwiPPfEU_1': _WeaviateUUIDInt('219354b1-dd2e-46c0-94cb-cdd51f915175'),
 '-OBCwiPPfEU_6': _WeaviateUUIDInt('27f967f2-3e9c-453d-8d21-378e4e15ffac'),
 '-OBCwiPPfEU_7': _WeaviateUUIDInt('332a363f-afcf-4fb7-9370-bbded23b8803'),
 '-OBCwiPPfEU_3': _WeaviateUUIDInt('59c539ac-8f95-4b13-bb6c-1fb323d7e64a'),
 '-OBCwiPPfEU_2': _WeaviateUUIDInt('746401bd-98c2-4494-ba08-d1b21d9abfc5'),
 '-OBCwiPPfEU_9': _WeaviateUUIDInt('77a3a7ae-8d77-4ae5-a0cd-705fb4286198'),
 '-OBCwiPPfEU_5': _WeaviateUUIDInt('803d2756-e3bd-44ef-88e6-43bc700be480')}
```

##### Finally you'll want to loop through your dataset, grab the doc_id value and expanded_content value and then update each object on the Weaviate cluster by using the uuid as found on the doc_id_cache

In [21]:
#Because this is not batched, expect this process to run for several minutes
for d in tqdm(data):
    doc_id = d['doc_id']
    expanded_content = d['expanded_content']
    uuid = doc_id_cache[doc_id]
    collection.data.update(uuid=uuid, properties={'expanded_content': expanded_content})

100%|██████████| 23123/23123 [50:09<00:00,  7.68it/s]
